# 观察数据

首先加载所有需要的包

In [58]:
import jieba
import pandas as pd
import numpy as np
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier



加载数据 

In [10]:
train = pd.read_csv('data/input/train.csv')
test = pd.read_csv('data/input/test_public.csv')
sample = pd.read_csv('data/input/submit_example.csv')

In [11]:
train.head()

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


In [13]:
test.head()

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


all1用来合并训练集和测试集中的content列

In [18]:
train_cutted = train[['content']]
test_cutted = test[['content']]
all1 = train_cutted.append(test_cutted,ignore_index=True)

In [19]:
all1.head()

,content
0,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。
1,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。
2,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...
3,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了
4,17价格忒高，估计也就是14-15左右。


预测标签转化

In [3]:
lbl_enc = preprocessing.LabelEncoder()
lbl_enc.fit(["动力", "价格", "内饰", "配置","安全性","外观","操控","油耗","空间","舒适性"])
y = lbl_enc.transform(train.subject.values)

# xtrain, xvalid, ytrain, yvalid = train_test_split(train.content.values, y, 
#                                                   stratify=y, 
#                                                   random_state=42, 
#                                                   test_size=0.1, shuffle=True)

In [4]:
y

array([0, 0, 0, ..., 1, 9, 2])

# 分词

In [32]:
import re
line = "Cats are smarter than dogs"
matchObj = re.match( r'(.*) are (.*?) .*', line, re.M|re.I)

In [33]:
matchObj

<_sre.SRE_Match object; span=(0, 26), match='Cats are smarter than dogs'>

In [21]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
all1["content_cutted"] = all1.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/lq/4cxvdwpx3xvbh36yldhr1cjw0000gn/T/jieba.cache
Loading model cost 0.835 seconds.
Prefix dict has been built succesfully.


In [34]:
all1["content_cutted"]

0        因为 森林 人 即将 换代 ， 这套 系统 没 必要 装在 一款 即将 换代 的 车型 上 ...
1        四驱 价格 貌似 挺 高 的 ， 高 的 可以 看齐 XC60 了 ， 看实车 前 脸 有点...
2        斯柯达 要说 质量 ， 似乎 比 大众 要 好 一点 ， 价格 也 低 一些 ， 用料 完全...
3        这 玩意 都 是 给 有钱 任性 又 不 懂车 的 土豪 用 的 ， 这 价格 换 一次 我...
4                   17 价格 忒 高 ， 估计 也 就是 14 - 15 左右 。      
5        我 开始 就是 荣放 2.5     森林 人 2.5 二选 一         荣放 主要...
6        唉 ， 这货 的 价格 死硬 死硬 的 ， 低配 版 优惠 1 万 据说 已经 罕有 了 。...
7          价格 的话 只能 说 一般般 吧 ， 太仓 前段时间 定 的 比 你 便宜 ！        
8                           听过 ， 价格 太贵 ， 但 一直 念念不忘        
9                                     恭喜 恭喜 ， 这个 优惠 不错 哦 ！
10                                   优惠 幅度 不小 了 ， 北京 优惠 八千
11            优惠 可以 了 ！ 购进 吧 ！ 买 了 不会 后悔 的 ！ 时间 可鉴 ！      
12                                       现在   什么 价   优惠 多少
13                            优惠 一 万一   送 贴膜 装甲 脚垫   铁西 庞大
14                    我 也 大连 的 ， 最近 也 考虑 入手 森林 人 ， 优惠 太小 了
15                               山东 威海 全系 这 才 优惠 3000 ， MD
16                                 下手 了 ， 豪导 特供 ， 优惠 1.6

In [22]:
all1.head()

,content,content_cutted
0,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,因为 森林 人 即将 换代 ， 这套 系统 没 必要 装在 一款 即将 换代 的 车型 上 ...
1,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,四驱 价格 貌似 挺 高 的 ， 高 的 可以 看齐 XC60 了 ， 看实车 前 脸 有点...
2,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,斯柯达 要说 质量 ， 似乎 比 大众 要 好 一点 ， 价格 也 低 一些 ， 用料 完全...
3,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,这 玩意 都 是 给 有钱 任性 又 不 懂车 的 土豪 用 的 ， 这 价格 换 一次 我...
4,17价格忒高，估计也就是14-15左右。,17 价格 忒 高 ， 估计 也 就是 14 - 15 左右 。


# 向量化

In [23]:
vect = CountVectorizer()
term_matrix_all = pd.DataFrame(vect.fit_transform(all1["content_cutted"]).toarray(), columns=vect.get_feature_names())

In [24]:
term_matrix_all.shape

(12311, 17823)

去掉停用词之后 再进行向量化

In [25]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
stop_words_file = "data/stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)
vect = CountVectorizer(stop_words=frozenset(stopwords))
term_matrix_all = pd.DataFrame(vect.fit_transform(all1["content_cutted"]).toarray(), columns=vect.get_feature_names())

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


去掉低频词和高频词后 再进行向量化

In [26]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
vect = CountVectorizer(max_df = max_df,
                       min_df = min_df,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))

term_matrix_all = pd.DataFrame(vect.fit_transform(all1["content_cutted"]).toarray(), columns=vect.get_feature_names())

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [27]:
term_matrix_all.shape

(12311, 6056)

缩减了单词的维度

# model

划分训练集和测试集 以及y

In [29]:
X_train = term_matrix_all.iloc[:9947]
Y_train = y
X_test  = term_matrix_all.iloc[9947:]
X_train.shape, Y_train.shape, X_test.shape

((9947, 6056), (9947,), (2364, 6056))

In [35]:
X_train.head()

,a180,a4,a4l,a6,a6l,a8,abs,ac,acc,act,...,黑屏,黑桶,黑色,默认,鼻子,齐亮,齐全,齿轮,齿轮油,龙膜
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


逻辑回归模型

In [36]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


81.19

In [37]:
Y_pred.shape

(2364,)

In [38]:
test.head()

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [39]:
Y_pred_name = lbl_enc.inverse_transform(Y_pred)

In [40]:
Y_pred_name

array(['价格', '价格', '价格', ..., '舒适性', '动力', '安全性'], dtype='<U3')

In [42]:
test_origin=test
test_origin['subject']=Y_pred_name

In [43]:
test_origin['sentiment_value']=0
test_origin['sentiment_word']='可以'

In [ ]:
test_origin = test_origin.drop(['content'], axis=1)

In [ ]:
test_origin.to_csv('data/output/logReg.csv',index=False)

In [ ]:
pre = pd.read_csv('data/output/LogReg.csv')

In [ ]:
pre.head()

## Random Forest

In [59]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred_RF = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

83.3

In [61]:
Y_pred_name_RF = lbl_enc.inverse_transform(Y_pred_RF)

In [62]:
test_origin_RF=test
test_origin_RF['subject']=Y_pred_name_RF
test_origin_RF['sentiment_value']=0
test_origin_RF['sentiment_word']='可以'

In [64]:
test_origin_RF = test_origin_RF.drop(['content'], axis=1)
test_origin_RF.to_csv('data/output/RandomForest.csv',index=False)

In [66]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred_SGD = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


80.24

In [67]:
# Linear SVC
from sklearn.svm import SVC, LinearSVC


linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


83.02

In [68]:
# Perceptron
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


79.08

In [69]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

51.21

In [72]:
from sklearn.neighbors import KNeighborsClassifier
# KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

73.39

# 情感分类

In [47]:
y_sentiment = train.sentiment_value.values

In [48]:
y_sentiment

array([ 0, -1,  1, ..., -1, -1,  0])

In [49]:
X_train_sentiment = term_matrix_all.iloc[:9947]
Y_train_sentiment = y_sentiment
X_test_sentiment  = term_matrix_all.iloc[9947:]
X_train_sentiment.shape, Y_train_sentiment.shape, X_test_sentiment.shape

((9947, 6056), (9947,), (2364, 6056))

In [50]:
logreg = LogisticRegression()
logreg.fit(X_train_sentiment, Y_train_sentiment)
Y_pred_sentiment = logreg.predict(X_test_sentiment)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


12.53

In [51]:
test_origin_sentiment=pd.read_csv('data/input/test_public.csv')

In [52]:
test_origin_sentiment.head()

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [53]:
test_origin_sentiment['subject']=Y_pred_name

In [54]:
test_origin_sentiment['sentiment_value']=Y_pred_sentiment

In [56]:
test_origin_sentiment

,content_id,content,subject,sentiment_value
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！,价格,1
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊,价格,0
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜,价格,0
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。,价格,1
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪,价格,0
5,cudSKGTyMLbIt8k3,2.0时尚优惠两万现金吗？还有其他赠品吗？,价格,0
6,U5RMQtopwFzDVxHm,27.5 相比较优惠的少,价格,0
7,Kh0ayJjuTrzwPAs1,综合优惠两万，不是现金优惠两万，送的垃圾东西都包含在内.大忽悠.,价格,0
8,YZovRtEAyQJgTi1I,差不多15000左右的优惠,价格,0
9,BeSIAUzoTWH9muYl,恭喜恭喜，这个配置性价比很高,配置,0


In [ ]:
test_origin_sentiment['sentiment_word']='可以'

In [ ]:
pred_sentiment = test_origin_sentiment.drop(['content'], axis=1)

In [ ]:
pred_sentiment.head()

In [ ]:
pred_sentiment.to_csv('data/output/logRegSentiment.csv',index=False)

In [ ]:
tes